In [1]:
%run config.ipynb

7.0.0
You are already using Cortex 7.0.0


ModuleNotFoundError: No module named 'sklearn'

In [4]:
from cortex import Cortex, Message
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
import json
import re
import time
import datetime
import dill
import sklearn

client = Cortex.client()
exp = client.experiment('flowers/iris-predict')

exp

In [5]:
run = exp.last_run() 

#to select a specific model you want uncomment the following (don't forget to comment out above) and put the exp id you want from above.
#run = exp.get_run(<ID>)

model = run.get_artifact('model')

print('Run id: %s \nModel name: "%s"' %(run.id, model))

Run id: an4y0fnc 
Model name: "DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')"


In [27]:
builder = client.builder()
builder.action('flowers/iris-predict')\
        .with_requirements(['tensorflow>=1.12,<2',
                            'keras>=2.2.4,<3'])\
        .from_model(model, model_type='tensorflow')\
        .with_base_image('continuumio/miniconda:4.5.12')\
        .daemon()\
        .build()

Building Cortex Action (daemon): flowers/iris-predict
Building Docker image private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:62stja1...
Step 1/12 : FROM continuumio/miniconda:4.5.12
Step 2/12 : WORKDIR /app
Removing intermediate container 40425f8b7d62
Step 3/12 : RUN conda config --add channels conda-forge
Removing intermediate container 0f45ec4b7c98
Step 4/12 : RUN conda install --yes flask gunicorn
Solving environment: ...working...
done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.5

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - flask
    - gunicorn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    itsdangerous-1.1.0         |             py_0          16 KB  conda-forge
    zlib-1.2.11     

BuilderException: Docker ERROR: The command '/bin/sh -c conda install --yes --file conda_requirements.txt' returned a non-zero code: 1

In [22]:
action = client.action('flowers/iris-predict')
action

Name,Version,Type,Kind,Image,Deployment Status,flowers/iris-predict,9,function,python:3,private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qpotji0,COMPLETED


In [23]:
params = {'columns': ['sepal_len','sepal_w','petal_len','petal_w'],
           'values': [4.9,3.1,1.5,0.2]}
frame = pd.DataFrame(params)
result = action.invoke(message=client.message(params))
print(result.payload)

2019-07-09 14:47:38,231 - INFO - cortex_client.client/client: Status: 500, Message: {"success":false,"error":"500 - {\"error\":{\"message\":\"No module named 'sklearn'\"}}"}


HTTPError: 500 Server Error: Internal Server Error for url: https://api.cortex.insights.ai/v3/actions/flowers/iris-predict/invoke

In [13]:
x = np.argmax(result.payload['values'][0])

iris_dict = {0:'Iris-setosa',1:'Iris-versicolor',2:'Iris-virginica'}

iris_dict[x]

NameError: name 'result' is not defined

**Create Skill Using the Action**

To create a skill from the action, call the builder to get a skill builder.

In [ ]:
skill_builder = builder.skill('flowers/iris-predict')\
     .title('Iris Flower Prediction')\
     .description('Measurement based prediction for the type of Iris flowers.')

A resulting schema is used by the skill builder to identify elements of the input. In this example, the schema describes the measurement values for parts of the Iris flower.

In [ ]:
schema_name = 'flowers/iris-input-schema'

builder.schema(schema_name)\
     .title('Iris Prediction Schema Instance')\
     .from_parameters([{'name': 'values', 'type': 'array', 'format': 'float32'}]).build()

skill_builder = skill_builder.input('iris-measurement').title('Iris Measurements')\
     .use_schema(schema_name)\
     .all_routing(action, 'measurement-based-prediction').build()

Specify the skill construction. The construction describes the skill including a name that matches the action name and a description of the output.

In [ ]:
skill_builder = skill_builder.output('measurement-based-prediction').title('Classification')\
    .parameter(name='IrisClass', type='array', format='float32').build()

With the skill builder fully specified, call to_camel so the skill’s contents can be reviewed.

In [ ]:
skill_builder.to_camel()

**Publish the Skill**

Invoke the builder to publish the skill. Upon publication, the skill can be used in Cortex Studio with other skills to compose agents.

In [ ]:
skill = skill_builder.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))